In [4]:
# import libraries

import requests
import io
import pandas as pd

import sqlalchemy, psycopg2
from sqlalchemy.types import FLOAT, TEXT, TIMESTAMP, BIGINT

from time import time

In [5]:
# download data and load as DataFrames

green_trip_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
# returns a compressed file, so include that in 'read_csv'
green_trip_df = pd.read_csv(io.BytesIO(requests.get(green_trip_url).content), compression='gzip')

zone_lookup_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
zone_lookup_df = pd.read_csv(io.StringIO(requests.get(zone_lookup_url).content.decode('utf-8')))

C:\Users\kingo\AppData\Local\Temp\ipykernel_7756\1753935676.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  green_trip_df = pd.read_csv(io.BytesIO(requests.get(green_trip_url).content), compression='gzip')


In [29]:
# establish connection to DB

user = 'postgres'
password = 'postgres'
host = 'localhost'
database = 'ny_taxi'
port = 5433

engine = sqlalchemy.create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

In [4]:
# use schema to determine column datatypes for db table
# print(pd.io.sql.get_schema(frame=green_trip_df, name="green_taxi", con=engine))

In [42]:
# create table with column names as in dataframe
green_trip_df.head(0).to_sql(name="green_taxi", con=engine, if_exists='replace')

green_trip_df.lpep_pickup_datetime = pd.to_datetime(green_trip_df.lpep_pickup_datetime)
green_trip_df.lpep_dropoff_datetime = pd.to_datetime(green_trip_df.lpep_dropoff_datetime)

started = time()

# load table with data from dataframe
green_trip_df.to_sql(
    name="green_taxi", con=engine, if_exists='append', chunksize=10000,
    dtype={
        "VendorID": FLOAT(53),
        "lpep_pickup_datetime": TIMESTAMP,
        "lpep_dropoff_datetime": TIMESTAMP,
        "store_and_fwd_flag": TEXT,
        "RatecodeID": FLOAT(53),
        "PULocationID": BIGINT,
        "DOLocationID": BIGINT,
        "passenger_count": FLOAT(53),
        "trip_distance": FLOAT(53),
        "fare_amount": FLOAT(53),
        "extra": FLOAT(53),
        "mta_tax": FLOAT(53),
        "tip_amount": FLOAT(53),
        "tolls_amount": FLOAT(53),
        "ehail_fee": FLOAT(53),
        "improvement_surcharge": FLOAT(53),
        "total_amount": FLOAT(53),
        "payment_type": FLOAT(53),
        "trip_type": FLOAT(53),
        "congestion_surcharge": FLOAT(53)
    }
)

finished = time()

print(f"Time taken: {(finished-started):.2f}")

Time taken: 143.71


In [15]:
mem_per_row = green_trip_df.memory_usage(index=True, deep=True).sum() / green_trip_df.shape[0]
chunk_size = 10000

print(f"Chunking consumes {((mem_per_row*chunk_size)/1000):.2f} kbs of memory per {chunk_size} rows.")

Chunking consumes 3411.22 kbs of memory per 10000 rows.


In [33]:
# load lookup data in one-go since rows are few
# create table with column names as in dataframe
zone_lookup_df.head(0).to_sql(name="lookup_table", con=engine, if_exists='replace')

# load table with data from dataframe
zone_lookup_df.to_sql(name="lookup_table", con=engine, if_exists='append')

265